## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [4]:
# requests 라이브러리 설치여부 확인 
!pip show requests

Name: requests
Version: 2.32.5
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: certifi, charset_normalizer, idna, urllib3
Required-by: anaconda-auth, anaconda-catalogs, anaconda-client, anaconda-project, conda, conda-build, conda-libmamba-solver, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, jupyterlab_server, panel, PyGithub, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract


In [5]:
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4


Name: beautifulsoup4
Version: 4.13.5
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: soupsieve, typing-extensions
Required-by: conda-build, nbconvert


In [6]:
# reqeusts, bs4 import
import requests
import bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup

In [7]:
# requests, bs4 버전 확인하기
print(f'requests 버전 = {requests.__version__}')
print(f'bs4 버전 = {bs4.__version__}')

requests 버전 = 2.32.5
bs4 버전 = 4.13.5


### 1. 뉴스 제목 추출하기

In [18]:
# IT/과학 뉴스 
#url = 'https://news.naver.com/section/105'

# dict타입으로 요청 파라미터 설정
req_param = {
    'sid': 105
}
# 
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

# 요청 헤더 설정 : 브라우저 정보 ( 사람처럼 보이게 하기 위함 )
req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3'
}

# requests 의 get() 함수 호출하기 
res = requests.get(url, headers=req_header)
print(res.status_code)
print(res.ok)
print(type(res))
#print(res.text)
# 응답(response)이 OK 이면
# 응답 (response)에서 text 추출
# BeautifulSoup 객체 생성  
if res.ok:
    soup = BeautifulSoup(res.text,'html.parser')
    print(len(soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")))
    # CSS 선택자를 사용해서 a tag 목록 가져오기
    a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")
    print(type(a_tags), type(a_tags[0])) # [Tag,Tag]
    # <a> 태그 리스트 순회하기    
    for a_tag in a_tags:
        title = a_tag.text.strip()
        link = a_tag['href']
        print(title, link)
else:
    # 응답(response)이 Error 이면 status code 출력    
    print(f'Error Code = {res.status_code}')   


https://news.naver.com/section/105
200
True
<class 'requests.models.Response'>
88
<class 'bs4.element.ResultSet'> <class 'bs4.element.Tag'>
LGU+, 작년 무선 가입자 3천만 돌파...서비스매출 4.1% 증가 https://n.news.naver.com/mnews/article/092/0002409310
 https://n.news.naver.com/mnews/article/comment/092/0002409310
구글, ‘고정밀 지도 반출’ 보완 서류 제출 https://n.news.naver.com/mnews/article/014/0005474401
 https://n.news.naver.com/mnews/article/comment/014/0005474401
휴네시온, 작년 매출 377억·영업익 48억…사상 최대 실적 https://n.news.naver.com/mnews/article/003/0013752851
 https://n.news.naver.com/mnews/article/comment/003/0013752851
넷마블, 2025년 역대 최대 매출…"신작 통해 올해 도약 전환점 마련" https://n.news.naver.com/mnews/article/092/0002409471
 https://n.news.naver.com/mnews/article/comment/092/0002409471
유명인 네이버 지식인 답변 '강제 노출'에 발칵[인터넷 와글와글] https://n.news.naver.com/mnews/article/057/0001934032
 https://n.news.naver.com/mnews/article/comment/057/0001934032
네이버 “커머스 최우선 과제는 배송 경쟁력 강화... N배송 적용 범위 3년 내 50%까지 확대”[컨콜] https://n.news.naver.com/mnews/article/3

### 1.1 뉴스제목 추출하는 함수 선언하기

In [26]:
import requests
from bs4 import BeautifulSoup

#section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}
section_dict = {'정치':100,'경제':101,'사회':102,'생활/문화':103,'세계':104,'IT/과학':105}

def print_news(section_name):  #print_new('생활/문화') 
    sid = section_dict.get(section_name,'정치')
    url = f'https://news.naver.com/section/{sid}'
    print(f'{section_name} 뉴스 {url}')
    req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3'
    }
    res = requests.get(url, headers=req_header)
    if res.ok:
        soup = BeautifulSoup(res.text,'html.parser')
        # CSS 선택자를 사용해서 a tag 목록 가져오기
        a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")
        print(type(a_tags), type(a_tags[0])) # [Tag,Tag]
        # <a> 태그 리스트 순회하기    
        for a_tag in a_tags:
            title = a_tag.text.strip()
            link = a_tag['href']
            print(title, link)
    else:
        # 응답(response)이 Error 이면 status code 출력    
        print(f'Error Code = {res.status_code}')

In [ ]:
print_news('세계')

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [ ]:
import requests
import os

# 육아일기 73회차
req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=812354&no=208&week=sun',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3'
}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/812354/208/20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/812354/208/20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/812354/208/20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_3.jpg'
]

for img_url in img_urls:
    # requests 의 get(url, headers) 함수 호출하기 
        res= requests.get(img_url, headers= req_header)
        print(res.status_code)
        # binary 응답 데이터 가져오기
        img_data = res.content
        # url에서 파일명만 추출하기
        file_name = os.path.basename(img_url)
        print(file_name)
        # binday data를 file에 write하기
        with open(file_name,'wb') as file:
                print(f'Writing to {file_name}({len(img_data):,} bytes)')
                file.write(img_data)

* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

webtoon_url = 'https://comic.naver.com/webtoon/detail?titleId=812354&no=208&week=thu'

req_header = {
    'referer':webtoon_url,
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3'
}

res = requests.get(webtoon_url, headers=req_header)
if res.ok:
    # .jpg 파일명을 추출해서 list에 저장하기
    soup = BeautifulSoup(res.text,'html.parser')
    print(len(soup.select("img[src*='IMAG01']")))
    img_tags = soup.select("img[src*='IMAG01']")
    # img_url_list = [] #list()
    # for img_tag in img_tags:
    #     img_url = img_tag['src']
    #     img_url_list.append(img_url)

    # List Comprehension        
    img_url_list2 = [img_tag['src'] for img_tag in img_tags]    
    print(img_url_list2[:2])

    imgdir_name = 'img'
    if not os.path.isdir(imgdir_name):
        os.mkdir(imgdir_name)

    for img_url in img_url_list2:
        # requests 의 get(url, headers) 함수 호출하기 
        res = requests.get(img_url, headers=req_header)
        # binary 응답 데이터 가져오기
        img_data = res.content    

        #img/xxxIMG01.jpg
        file_path = os.path.join(imgdir_name,os.path.basename(img_url))
        # binday data를 file에 write하기
        with open(file_path,'wb') as file:
            print(f'Writing to {file_path}({len(img_data):,} bytes)')
            file.write(img_data)        
else:
    print(f'Error Code = {res.status_code}')

In [ ]:
###리팩토링 된 코드
import requests
from bs4 import BeautifulSoup
import os

# 기본 설정
url = 'https://comic.naver.com/webtoon/detail?titleId=833255&no=3&week=tue'
req_header = {'referer': url}
imgdir_name = 'img'

# 이미지 저장 폴더가 없으면 생성 (sub 디렉토리도 생성- makedirs가 생성해줌.)
os.makedirs(imgdir_name, exist_ok=True)

# 웹 페이지 요청 및 확인
res = requests.get(url)
if not res.ok:
    print(f'Error Code = {res.status_code}')
    exit()

# 이미지 URL 추출
soup = BeautifulSoup(res.text, 'html.parser')
img_url_list = [img_tag['src'] for img_tag in soup.select("img[src*='IMAG01']")]

# 이미지 다운로드
for img_url in img_url_list:
    res = requests.get(img_url, headers=req_header)
    if res.ok:
        img_data = res.content
        file_path = os.path.join(imgdir_name, os.path.basename(img_url))
        with open(file_path, 'wb') as file:
            print(f'Writing to {file_path} ({len(img_data):,} bytes)')
            file.write(img_data)
    else:
        print(f'Error Code = {res.status_code} for {img_url}')

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [ ]:
import requests

upload_files = {
    'img1': open('img/f1.jpg','rb'),
    'img2': open('img/f2.jpg','rb'),
}
print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.
res= requests.post(url, files= upload_files)
print(res.status_code)
print(res.json()['files']['img1'])


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
CLIENT_ID = os.getenv("CLIENT_ID")
print(CLIENT_ID[:4])

CLIENT_SECRET = os.getenv("CLIENT_SECRET")
print(CLIENT_SECRET[:4])

k8Kd
DLKL


In [9]:
# 캡차 키 발급 요청
import os
import sys
import urllib.request

code = "0"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",CLIENT_ID)
request.add_header("X-Naver-Client-Secret",CLIENT_SECRET)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"key":"UkAUQ9DcPBzjt7s0"}


In [11]:
import urllib.request

key = "UkAUQ9DcPBzjt7s0" # 캡차 Key 값
url = "https://openapi.naver.com/v1/captcha/ncaptcha.bin?key=" + key
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",CLIENT_ID)
request.add_header("X-Naver-Client-Secret",CLIENT_SECRET)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    print("캡차 이미지 저장")
    response_body = response.read()
    with open('img/captcha.jpg', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)

캡차 이미지 저장


In [12]:
#  사용자 입력값 검증 요청
import requests

code = "1"
key = "UkAUQ9DcPBzjt7s0"
value = "N9HLE"
req_header = {
    "X-Naver-Client-Id": CLIENT_ID,
    "X-Naver-Client-Secret": CLIENT_SECRET
}
req_param = {
    "code": code, 
    "key": key,
    "value": value
}
url = "https://openapi.naver.com/v1/captcha/nkey"

try:
    res = requests.get(url, headers= req_header, params= req_param)
    #4xx,5xx 오류 발생시 예외를 발생시킴
    res.raise_for_status()

    print(res.text)
except requests.exceptions.RequestException as e:
    print(f'Status Code : {res.status_code}')
    print(f'Error 발생 : {e} ')

{"result":true,"responseTime":33.97}


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

### 5. 블로그 검색하기

In [ ]:
import requests
from pprint import pprint

headers = {
    'X-Naver-Client-Id': CLIENT_ID,
    'X-Naver-Client-Secret': CLIENT_SECRET,
}

payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'

# requests get(url, params, headers) 요청 
res = requests.get(url, params=payload, headers=headers)
# json() 함수로 응답 결과 가져오기 [{},{},{}]
print(len(res.json()['items'])) 
pprint(res.json()['items'])

items_data = res.json()['items']
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'

In [22]:
#data/blog.json 파일 생성하기
import json
with open('data/blog.json', 'w', encoding='utf-8') as file:
    json.dump(items_data, file)